In [69]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument('--image_size', type=int, default=64)
parser.add_argument('--batch_size', type=int, default=64) # DCGAN paper original value used 128
parser.add_argument('--n_colors', type=int, default=3)
parser.add_argument('--z_size', type=int, default=100) # DCGAN paper original value
parser.add_argument('--G_h_size', type=int, default=128, help='Number of hidden nodes in the Generator. Too small leads to bad results, too big blows up the GPU RAM.') # DCGAN paper original value
parser.add_argument('--D_h_size', type=int, default=128, help='Number of hidden nodes in the Discriminator. Too small leads to bad results, too big blows up the GPU RAM.') # DCGAN paper original value
parser.add_argument('--lr_D', type=float, default=.00005, help='Discriminator learning rate') # 1/4 of DCGAN paper original value
parser.add_argument('--lr_G', type=float, default=.0002, help='Generator learning rate') # DCGAN paper original value
parser.add_argument('--n_epoch', type=int, default=1000)
parser.add_argument('--beta1', type=float, default=0.5, help='Adam betas[0], DCGAN paper recommends .50 instead of the usual .90')
parser.add_argument('--SELU', type=bool, default=False, help='Using scaled exponential linear units (SELU) which are self-normalizing instead of ReLU with BatchNorm. This improves stability.')
parser.add_argument('--seed', type=int)
parser.add_argument('--input_folder', default='./cats_bigger_than_64x64/', help='input folder')
parser.add_argument('--output_folder', default='Output/DCGAN', help='output folder')
parser.add_argument('--G_load', default='', help='Full path to Generator model to load (ex: /home/output_folder/run-5/models/G_epoch_11.pth)')
parser.add_argument('--D_load', default='', help='Full path to Discriminator model to load (ex: /home/output_folder/run-5/models/D_epoch_11.pth)')
parser.add_argument('--cuda', type=bool, default=True, help='enables cuda')
parser.add_argument('--n_gpu', type=int, default=1, help='number of GPUs to use')
parser.add_argument('--n_workers', type=int, default=2, help='Number of subprocess to use to load the data. Use at least 2 or the number of cpu cores - 1.')
parser.add_argument('--weight_decay', type=float, default=0, help='L2 regularization weight. Greatly helps convergence but leads to artifacts in images, not recommended.')
parser.add_argument('--gen_extra_images', type=int, default=0, help='Every epoch, generate additional images with "batch_size" random fake cats.')
# param = parser.parse_args()

## Imports, type=int, default=0, help='Every epoch, generate additional images with "batch_size" random fake cats.')
# param = parser.parse_args()
args, unknown = parser.parse_known_args()

In [70]:
# wandb.finish()

In [71]:
wb_run = wandb.init(project="cat-generation-DCGAN",config={"architecture": "DCGA"})
wandb.config.update(args) # adds all of the arguments as config variables
param = wandb.config

print(param)

{'architecture': 'DCGA', 'image_size': 64, 'batch_size': 64, 'n_colors': 3, 'z_size': 100, 'G_h_size': 128, 'D_h_size': 128, 'lr_D': 5e-05, 'lr_G': 0.0002, 'n_epoch': 1000, 'beta1': 0.5, 'SELU': False, 'seed': None, 'input_folder': './cats_bigger_than_64x64/', 'output_folder': 'Output/DCGAN', 'G_load': '', 'D_load': '', 'cuda': True, 'n_gpu': 1, 'n_workers': 2, 'weight_decay': 0, 'gen_extra_images': 0}


In [72]:
# %tb
# wandb.config

In [73]:
# !pip install wandb -qqq
# import wandb

In [74]:
# wandb.login()

In [75]:
# !wandb login --relogin


In [76]:
## Imports

import time
start = time.time()
import os

run=0
base_dir = f"{param.output_folder}/run-{run}/"
while os.path.exists(base_dir):
    run+=1
    base_dir = f"{param.output_folder}/run-{run}"
os.makedirs(base_dir)
# logs_dir = f"{base_dir}/logs"
# os.makedirs(logs_dir)
# os.makedirs(f"{base_dir}/images")
os.makedirs(f"{base_dir}/models")
# if param.gen_extra_images>0:
#     os.makedirs(f"{base_dir}/images/extra/")

In [77]:
# param

In [78]:

import torch
import torch.autograd as autograd
from torch.autograd import Variable

# from tensorboard_logger import configure,log_value
# configure(logs_dir,flush_secs=5)

import torchvision
import torchvision.datasets as dset
import torchvision.transforms as transf
import torchvision.models as models
import torchvision.utils as vutils
from torch.utils.data import Dataset
from natsort import natsorted
# from PIL import Image
import PIL
import imageio as iio

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if param.cuda:
    import torch.backends.cudnn as cudnn
    cudnn.benchmark = True
    
from IPython.display import Image
to_img = transf.ToPILImage()

import math

import random
param.seed = param.seed or random.randint(1,10000)
print(f"Random Seed: {param.seed}")
wandb.log({"Random": param.seed})
# print(f"Random Seed: {param.seed}",file=log_output)
random.seed(param.seed)
torch.manual_seed(param.seed)

if param.cuda:
    torch.cuda.manual_seed_all(param.seed)
    
## Transforming images
trans = transf.Compose([
    transf.Resize((param.image_size,param.image_size)),
    # This makes it into [0,1]
    transf.ToTensor(),
    # This makes it into [-1,1] so tanh will work properply
    transf.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
])

## Importing dataset
# data = dset.ImageFolder(root=param.input_folder, transform=trans)

# Loading data in batch
# dataset = torch.utils.data.DataLoader(data,batch_size=param.batch_size,shuffle=True,num_workers=param.n_workers)

class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform):
        self.main_dir = main_dir
        self.transform = transform
        all_imgs = os.listdir(main_dir)
        self.total_imgs = natsorted(all_imgs)

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        image = PIL.Image.open(img_loc).convert("RGB")
        # image = iio.imread(img_loc)
        tensor_image = self.transform(image)
        return tensor_image
    
my_dataset = CustomDataSet(param.input_folder, transform=trans)
dataset = torch.utils.data.DataLoader(my_dataset , batch_size=param.batch_size, shuffle=True, 
                               num_workers=param.n_workers)

Random Seed: 6600


In [79]:
# param.seed
# !pip install natsort

In [80]:
## Models
# The number of layers is implicitly determined by the image size
# image_size = (4,8,16,32,64,128,256, 512, 1024) leads to n_layers = (1,2,3,4,5,6,7,8,9)

# The more layers the bigger the neural get so it's best to decrease D_h_size and D_h_size when the image input is bigger

# DCGAN generator
class DCGAN_G(torch.nn.Module):
    def __init__(self):
        super(DCGAN_G,self).__init__()
        main = torch.nn.Sequential()
        
        # We need to know how many layers we will use at the beginning
        mult = param.image_size // 8
        
        
        ### Start block
        # Z_size random numbers
        main.add_module('Start-ConvTranspose2d',torch.nn.ConvTranspose2d(param.z_size,param.G_h_size*mult,kernel_size=4,stride=1,padding=0,bias=False))
        if param.SELU:
            main.add_module('Start-SELU',torch.nn.SELU(inplace=True))
        else:
            main.add_module('Start-BatchNorm2d',torch.nn.BatchNorm2d(param.G_h_size*mult))
            main.add_module('Start-ReLU',torch.nn.ReLU())
        # Size = (G_h_size * mult)x 4 x4
        
        ### Middle block (Done until we reach  ? x image_size/2 x image_size/2)
        i=1
        while mult>1:
            main.add_module('Middle-ConvTranspose2d [%d]'%i, torch.nn.ConvTranspose2d(param.G_h_size*mult,param.G_h_size*(mult//2),kernel_size=4,stride=2,padding=1,bias=False))
            if param.SELU:
                main.add_module('Middle-SELU [%d]'%i,torch.nn.SELU(inplace=True))
            else:
                main.add_module('Middle-BatchNorm2d [%d]'%i,torch.nn.BatchNorm2d(param.G_h_size*(mult//2)))
                # main.add_module('Middle_ReLU [$d]'%i,torch.nn.SELU(inplace=True))
                main.add_module('Middle-ReLU [%d]' % i, torch.nn.ReLU(inplace=True))
            # Size = (G_h_size * (mult/(2*i)))x8x8
            mult = mult // 2
            i+=1
        
        ### End block
        # Size = G_h_size/2 x image_size/2
        main.add_module('End-ConvTransposed2d', torch.nn.ConvTranspose2d(param.G_h_size,param.n_colors,kernel_size=4,stride=2,padding=1,bias=False))
        main.add_module('End-Tanh',torch.nn.Tanh())
        # Size = n_colors x image_size x image_size
        self.main = main
    
    def forward(self,input):
        if isinstance(input.data,torch.cuda.FloatTensor) and param.n_gpu>1:
            output = torch.nn.parallel.data_parallel(self.main,input,range(param.n_gpu))
        else:
            output = self.main(input)
        return output
# DCGAN discriminator ( using somewhat the reverse of the generator)
class DCGAN_D(torch.nn.Module):
    def __init__(self):
        super(DCGAN_D,self).__init__()
        main = torch.nn.Sequential()
        
        ### start block
        # size = n_colors x image_size x image_size
        main.add_module('Start-conv2d',torch.nn.Conv2d(param.n_colors,param.D_h_size,kernel_size=4,stride=2,padding=1,bias=False))
        if param.SELU:
            main.add_module('Start-SELU',torch.nn.SELU(inplace=True))
        else:
            main.add_module('Start-LeakyReLU',torch.nn.LeakyReLU(0.2,inplace=True))
        image_size_new = param.image_size // 2
        # Size = D_h_size x image_size/2 x image_size/2
        
        ### Middle block (Done until we reach ? x 4 x4)
        mult = 1
        i=0
        while image_size_new>4:
            main.add_module('Middle-Conv2d [%d]'%i,torch.nn.Conv2d(param.D_h_size*mult,param.D_h_size*(2*mult),kernel_size=4,stride=2,padding=1,bias=False))
            if param.SELU:
                main.add_module('Middle-SELU [%d]'%i,torch.nn.SELU(inplace=True))
            else:
                main.add_module('Middle-BatchNorm2d [%d]'%i,torch.nn.BatchNorm2d(param.D_h_size*(2*mult)))
                main.add_module('Middle-LeakyReLU [%d]'%i,torch.nn.LeakyReLU(0.2,inplace=True))
            # Size = (D_h_size*(2*i)) x image_size/(2*i) x image_size/(2*i)
            image_size_new = image_size_new // 2
            mult *=2
            i +=1
            
        ## End Block
        # size = (D_h_size*mult) x 4 x4
        main.add_module('End-Conv2d',torch.nn.Conv2d(param.D_h_size*mult,1,kernel_size=4,stride=1,padding=0,bias=False))
        main.add_module('Sigmoid',torch.nn.Sigmoid())
        # size = 1 x1 x1 (Is a real cat or not)
        self.main = main
    def forward(self,input):
        if isinstance(input.data,torch.cuda.FloatTensor) and param.n_gpu>1:
            output = torch.nn.parallel.data_parallel(self.main,input,range(param.n_gpu))
        else:
            output = self.main(input)
            
            # Convert from 1 x 1 x 1 to 1 so that we can compare to given label (cat or not?)
        return output.view(-1)
        
## weights init function, DCGAN use 0.02 std
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0,0.02)
    elif classname.find('BatchNorm') != -1:
        # estimate variance, must be around 1
        m.weight.data.normal_(1.0,0.02)
        # estimate mean, must be around 0
        m.bias.data.fill_(0)
        
# Init
G = DCGAN_G()
D = DCGAN_D()

G.apply(weights_init)
D.apply(weights_init)

# Load existing models
if param.G_load !='':
    G.load_state_dict(torch.load(param.G_load))
if param.D_load !='':
    D.load_state_dict(torch.load(param.D_load))
    
print(G)
print(D)

# criterion
criterion = torch.nn.BCELoss()

# # soon to be variables
# x = torch.FloatTensor(param.batch_size,param.n_colors,param.image_size,param.image_size)
# y = torch.FloatTensor(param.batch_size)
# z = torch.FloatTensor(param.batch_size,param.z_size,1,1)
# # This is to see during training, size and values won't change
z_test = torch.FloatTensor(param.batch_size,param.z_size,1,1).normal_(0,1)

# Everthing cuda

G = G.to(device)
D = D.to(device)
criterion = criterion.to(device)
z_test = z_test.to(device)
    
# Now Variables
# x = Variable(x)
# y = Variable(y)
# z = Variable(z)
# z_test = Variable(z_test)

# Based on DCGAN paper, they found using betas[0]=0.50 better.
# betas[0] represent is the weight given to the previous mean of the gradient
# betas[1] is the weight given to the previous variance of the gradient
optimizerD = torch.optim.Adam(D.parameters(),lr=param.lr_D, betas=(param.beta1,0.999),weight_decay=param.weight_decay)
optimizerG = torch.optim.Adam(G.parameters(),lr=param.lr_G, betas=(param.beta1,0.999),weight_decay=param.weight_decay)


DCGAN_G(
  (main): Sequential(
    (Start-ConvTranspose2d): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (Start-BatchNorm2d): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Start-ReLU): ReLU()
    (Middle-ConvTranspose2d [1]): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (Middle-BatchNorm2d [1]): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Middle-ReLU [1]): ReLU(inplace=True)
    (Middle-ConvTranspose2d [2]): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (Middle-BatchNorm2d [2]): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (Middle-ReLU [2]): ReLU(inplace=True)
    (Middle-ConvTranspose2d [3]): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (Middle-BatchNorm2d [3]): BatchNorm2d(128, eps=1e-05, 

In [81]:
def log_images(images):
    img_grid = torchvision.utils.make_grid(images)
    img = wandb.Image(img_grid)
    wandb.log({"examples": img})

In [82]:
## fitting model

for epoch in range(param.n_epoch):
    # Fake images saved
    fake_test = G(z_test)
    log_images(fake_test)        
    for i, data_batch in enumerate(dataset,0):
        # Update D network
        
        # for p in D.parameters():
        #     p.requires_grad = True
        
        # Train with real data
        D.zero_grad()
        # We can ignore labels since they are all cats
        images = data_batch
        # Mostly necessary for the last one because if the N might not be a multiple of batch_size
        current_batch_size = images.size(0)
        # if param.cuda:
        images= images.to(device)
        # Transfer batch of images to x
        # y is now a vector of size current_batch_size filled with 1
        y = torch.ones(current_batch_size).to(device)
        y_pred = D(images)
        errD_real = criterion(y_pred,y)
        errD_real.backward()
        # Var has data and gradient element, we keep the mean of the data element
        D_real = torch.mean(y_pred)
        
        # Train with fake data
        z = torch.randn(current_batch_size,param.z_size,1,1).to(device)
        x_fake = G(z)
        y = torch.zeros(current_batch_size).to(device)
        # y.data.resize_(current_batch_size).fill_(0)
        # Detach y_pred from the neural network G and put it inside D
        y_pred_fake = D(x_fake.detach())
        errD_fake = criterion(y_pred_fake,y)
        errD_fake.backward()
        D_fake = torch.mean(y_pred_fake)
        errD = errD_real + errD_fake
        optimizerD.step()
        
        # Update G net work
        # make it tiny bit faster
        # for p in D.parameters():
        #     p.requires_grad = False
            
        G.zero_grad()
        # Generator wants to foll discriminaor so it wants to minimize loss of discriminator assuming label is True
        y = torch.ones(current_batch_size).to(device)
        y_pred_fake = D(x_fake)
        errG = criterion(y_pred_fake,y)
        errG.backward(retain_graph=True)
        D_G = torch.mean(y_pred_fake)
        optimizerG.step()
        
        current_step = i + epoch*len(dataset)
        # Log results so we can see them in TensorBoard after
        metrics = {"train/errD": errD.item(), 
                   "train/errG": errG.item()}
        wandb.log(metrics)
        if i%50 ==0:
            end = time.time()
            fmt = '[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f/%.4f time:%.4f'
            s = fmt%(epoch,param.n_epoch,i,len(dataset),errD.item(),errG.item(),D_real,D_fake,D_G,end-start)
            print(s)
            # wandb.log({"train/logs":s})
            
    # save every epoch
    fmt = '%s/run-%d/models/%s_epoch_%d.pth'
    if epoch%25==0:
        torch.save(G.state_dict(),fmt%(param.output_folder, run, 'G',epoch))
        torch.save(D.state_dict(), fmt%(param.output_folder, run,'D',epoch))
        fmt = '%s/run-%d/models/'
        trained_model = wandb.Artifact("DCGAN_model_epoch_{}".format(epoch),type="model")
        trained_model.add_dir(fmt%(param.output_folder, run))
        wb_run.log_artifact(trained_model)

wandb.finish()
        

[0/1000][0/146] Loss_D: 2.8515 Loss_G: 1.8969 D(x): 0.1522 D(G(z)): 0.1401/0.2353 time:7.0482
[0/1000][50/146] Loss_D: 2.9082 Loss_G: 12.4335 D(x): 0.5186 D(G(z)): 0.4457/0.0000 time:37.5775
[0/1000][100/146] Loss_D: 1.1136 Loss_G: 10.7004 D(x): 0.7372 D(G(z)): 0.2190/0.0000 time:68.2946


wandb: Adding directory to artifact (./Output/DCGAN/run-3/models)... Done. 0.5s


[1/1000][0/146] Loss_D: 3.4345 Loss_G: 15.9139 D(x): 0.7556 D(G(z)): 0.9014/0.0000 time:97.7495
[1/1000][50/146] Loss_D: 0.5629 Loss_G: 9.0409 D(x): 0.8275 D(G(z)): 0.0223/0.0002 time:128.7325
[1/1000][100/146] Loss_D: 0.8668 Loss_G: 5.4489 D(x): 0.7295 D(G(z)): 0.1301/0.0068 time:159.7860
[2/1000][0/146] Loss_D: 0.6211 Loss_G: 6.9942 D(x): 0.8008 D(G(z)): 0.0876/0.0027 time:188.4672
[2/1000][50/146] Loss_D: 1.6359 Loss_G: 1.6061 D(x): 0.4961 D(G(z)): 0.3683/0.2964 time:219.5728
[2/1000][100/146] Loss_D: 1.2082 Loss_G: 2.0269 D(x): 0.5881 D(G(z)): 0.3280/0.1664 time:250.6816
[3/1000][0/146] Loss_D: 1.2674 Loss_G: 1.8053 D(x): 0.6714 D(G(z)): 0.5145/0.2107 time:279.4042
[3/1000][50/146] Loss_D: 1.6817 Loss_G: 1.2246 D(x): 0.4649 D(G(z)): 0.4532/0.3186 time:310.5298
[3/1000][100/146] Loss_D: 1.3285 Loss_G: 1.6198 D(x): 0.4930 D(G(z)): 0.3135/0.2391 time:341.6380
[4/1000][0/146] Loss_D: 1.5141 Loss_G: 1.0743 D(x): 0.5572 D(G(z)): 0.4971/0.3845 time:370.3575
[4/1000][50/146] Loss_D: 1.2195

wandb: Adding directory to artifact (./Output/DCGAN/run-3/models)... Done. 0.8s


[26/1000][0/146] Loss_D: 1.4293 Loss_G: 0.9344 D(x): 0.5699 D(G(z)): 0.5419/0.4094 time:2374.9097
[26/1000][50/146] Loss_D: 1.5234 Loss_G: 0.9261 D(x): 0.4925 D(G(z)): 0.4956/0.4187 time:2406.0505
[26/1000][100/146] Loss_D: 1.5055 Loss_G: 0.8748 D(x): 0.4447 D(G(z)): 0.4594/0.4374 time:2437.1806
[27/1000][0/146] Loss_D: 1.3748 Loss_G: 1.0019 D(x): 0.5440 D(G(z)): 0.4933/0.3836 time:2465.9440
[27/1000][50/146] Loss_D: 1.3047 Loss_G: 1.0502 D(x): 0.5356 D(G(z)): 0.4633/0.3703 time:2497.1144
[27/1000][100/146] Loss_D: 1.3259 Loss_G: 1.1327 D(x): 0.5359 D(G(z)): 0.4703/0.3384 time:2528.2547
[28/1000][0/146] Loss_D: 1.4238 Loss_G: 1.1441 D(x): 0.5378 D(G(z)): 0.5307/0.3490 time:2557.0102
[28/1000][50/146] Loss_D: 1.3723 Loss_G: 1.0364 D(x): 0.5258 D(G(z)): 0.4934/0.3697 time:2588.1640
[28/1000][100/146] Loss_D: 1.4287 Loss_G: 1.0644 D(x): 0.4889 D(G(z)): 0.4665/0.3630 time:2619.2937
[29/1000][0/146] Loss_D: 1.4083 Loss_G: 0.9577 D(x): 0.6101 D(G(z)): 0.5672/0.4025 time:2648.0191
[29/1000][5

...

wandb: Adding directory to artifact (./Output/DCGAN/run-3/models)... Done. 2.2s


[951/1000][0/146] Loss_D: 0.0045 Loss_G: 8.0707 D(x): 0.9997 D(G(z)): 0.0042/0.0016 time:86480.3187
[951/1000][50/146] Loss_D: 0.0037 Loss_G: 8.4432 D(x): 0.9985 D(G(z)): 0.0022/0.0012 time:86511.2995
[951/1000][100/146] Loss_D: 0.0071 Loss_G: 8.8200 D(x): 0.9937 D(G(z)): 0.0007/0.0011 time:86542.2349
[952/1000][0/146] Loss_D: 0.0030 Loss_G: 7.8050 D(x): 0.9989 D(G(z)): 0.0019/0.0013 time:86570.8930
[952/1000][50/146] Loss_D: 0.0042 Loss_G: 7.7376 D(x): 0.9982 D(G(z)): 0.0024/0.0015 time:86601.8396
[952/1000][100/146] Loss_D: 0.0069 Loss_G: 8.0884 D(x): 0.9976 D(G(z)): 0.0043/0.0014 time:86632.7965
[953/1000][0/146] Loss_D: 0.0024 Loss_G: 8.6369 D(x): 0.9995 D(G(z)): 0.0019/0.0008 time:86661.4356
[953/1000][50/146] Loss_D: 0.0033 Loss_G: 8.0919 D(x): 0.9981 D(G(z)): 0.0014/0.0012 time:86692.4162
[953/1000][100/146] Loss_D: 0.0046 Loss_G: 8.3495 D(x): 0.9996 D(G(z)): 0.0042/0.0015 time:86723.3946
[954/1000][0/146] Loss_D: 0.0030 Loss_G: 8.8319 D(x): 0.9995 D(G(z)): 0.0025/0.0008 time:86

wandb: Adding directory to artifact (./Output/DCGAN/run-3/models)... Done. 2.3s


[976/1000][0/146] Loss_D: 0.0052 Loss_G: 8.4273 D(x): 0.9976 D(G(z)): 0.0027/0.0009 time:88748.9895
[976/1000][50/146] Loss_D: 0.0043 Loss_G: 10.0102 D(x): 0.9967 D(G(z)): 0.0009/0.0008 time:88779.9748
[976/1000][100/146] Loss_D: 0.0044 Loss_G: 9.4233 D(x): 0.9980 D(G(z)): 0.0024/0.0008 time:88810.9585
[977/1000][0/146] Loss_D: 0.0061 Loss_G: 8.7236 D(x): 0.9959 D(G(z)): 0.0019/0.0018 time:88839.5931
[977/1000][50/146] Loss_D: 0.0012 Loss_G: 9.4367 D(x): 0.9995 D(G(z)): 0.0006/0.0005 time:88870.5772
[977/1000][100/146] Loss_D: 0.0052 Loss_G: 10.5868 D(x): 0.9971 D(G(z)): 0.0023/0.0005 time:88901.5569
[978/1000][0/146] Loss_D: 0.0630 Loss_G: 9.6902 D(x): 0.9806 D(G(z)): 0.0061/0.0026 time:88930.1791
[978/1000][50/146] Loss_D: 0.0997 Loss_G: 16.1706 D(x): 0.9647 D(G(z)): 0.0353/0.0016 time:88961.1528
[978/1000][100/146] Loss_D: 0.0447 Loss_G: 10.1146 D(x): 0.9893 D(G(z)): 0.0306/0.0013 time:88992.1552
[979/1000][0/146] Loss_D: 0.0214 Loss_G: 9.2589 D(x): 0.9990 D(G(z)): 0.0170/0.0026 tim

Random,▁
train/errD,██▆▅▃▂▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/errG,▁▁▁▂▃▃▃▄▄▄▅▅▄▅▅▅▆▄▃▆▅▆▅▆▇▅▆▇▆▆▆▇▇█▆▆▆█▆▇
Random,6600
train/errD,0.00272
train/errG,8.34883


In [24]:
len(dataset)

146

In [22]:
# !pip install wandb -qqq
import wandb

In [20]:
%tensorboard --logdir Output/DCGAN/run-42/logs/log.txt
# !pip uninstall tb-nightly tensorboardX tensorboard
# !pip install tensorboard
# !kill 37015

In [13]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [60]:
# !tensorboard --logdir "./cats_bigger_than_64x64/"
# %tensorboard --logdir "GANS/cat/Output/DCGAN/run-42/logs/log.txt"
# %load_ext tensorboard
# %tensorboard --logdir=./Output/DCGAN/run-42/logs
# !pip install TensorBoard==1.15.0
# !kill 5926
# import cv2
# import os

# image_folder = 'Output/DCGAN/run-42/images'
# video_name = 'video.avi'

# images = [img for img in os.listdir(image_folder) if img.endswith(".png")]
# frame = cv2.imread(os.path.join(image_folder, images[0]))
# height, width, layers = frame.shape

# video = cv2.VideoWriter(video_name, 0, 1, (width,height))

# for image in images:
#     video.write(cv2.imread(os.path.join(image_folder, image)))

# cv2.destroyAllWindows()
# video.release()


import os
import cv2
# from PIL import Image
# os.chdir("/home/ganesh/Desktop/video")
total_width=0
total_height=0
for file in os.listdir('Output/DCGAN/run-42/images'):
    im=PIL.Image.open(os.path.join("./Output/DCGAN/run-42/images/",file))
    width,height=im.size
    total_width+=width
    total_height+=height
numofimages=len(os.listdir('Output/DCGAN/run-42/images'))
mean_width=int(total_width/numofimages)  #calculating the width for each image
mean_height=int(total_height/numofimages)  #calculating the height for each image 
for file in os.listdir('Output/DCGAN/run-42/images'):
    if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png"):
        im=PIL.Image.open(os.path.join("./Output/DCGAN/run-42/images/",file))
        imResize=im.resize((mean_width,mean_height),PIL.Image.ANTIALIAS)
        imResize.save(file,'JPEG',quality=95)



In [51]:
# !pip uninstall opencv-python; pip install opencv-python 

In [58]:
!rm -r ./Output/DCGAN/run-42/images/.ipynb_checkpoints

rm: cannot remove './Output/DCGAN/run-42/images/.ipynb_checkpoints': No such file or directory


In [7]:
import numpy

# def generate_video():
#     image_folder = '.' # Use the folder
#     video_name = 'mygeneratedvideo.avi'
#     # os.chdir("/home/ganesh/Desktop/video")
#     images = [img for img in os.listdir(image_folder) if img.endswith(".jpg") or
#              img.endswith(".jpeg") or img.endswith("png")]
   
#     fourcc = cv2.VideoWriter_fourcc(*'DIVX') 

#     # Array images should only consider 
#     # the image files ignoring others if any 
    
#     pil_image = PIL.Image.open(os.path.join(image_folder, images[0])).convert('RGB') 
#     # pil_image = PIL.Image.open('Image.jpg').convert('RGB') 
#     frame = numpy.array(pil_image) 
#     # Convert RGB to BGR 
#     frame = frame[:, :, ::-1].copy() 
    
    # setting the frame width, height width 
#     # the width, height of first image 
#     print(frame)
#     height, width, layers = frame.shape

#     video = cv2.VideoWriter(video_name, fourcc, 1, (width, height))

#     # Appending the images to the video one by one 
#     for image in images:
#         video.write(cv2.imread(os.path.join(image_folder,image)))
#         break
        
#     # Deallocating memories taken for window creation 
#     cv2.destroyAllWindows()
#     video.release()  # releasing the video generated 
    
# generate_video()

from moviepy.editor import *
from pathlib import Path

img_clips = []
path_list=[]

#accessing path of each image
for image in sorted(os.listdir('Output/DCGAN/run-42/images/')):
    if image.endswith(".png"):
        path_list.append(os.path.join('Output/DCGAN/run-42/images/', image))

#creating slide for each image
for img_path in path_list:
    slide = ImageClip(img_path,duration=2)
    img_clips.append(slide)

#concatenating slides
video_slides = concatenate_videoclips(img_clips, method='compose')
#exporting final video
video_slides.write_videofile("output_video.mp4", fps=24*2)


Moviepy - Building video output_video.mp4.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4


In [75]:
# !pip install moviepy
# !pip install moviepy --upgrade

In [67]:
# !rm fake_samples_e*

In [20]:
# !pip uninstall -y tensorboard tb-nightly
# !pip install tb-nightly  # must have at least tb-nightly==2.5.0a20210316
# tensorboard --logdir log.txt --load_fast=true

SyntaxError: invalid syntax (3091424231.py, line 3)

In [ ]:
img for img in os.listdir(image_folder) if img.endswith(".png")

In [44]:
# os.listdir('Output/DCGAN/run-42/images')

In [32]:
!cp -r cats_bigger_than_64x64/.ipynb_checkpoints .

ls: invalid option -- 'e'
Try 'ls --help' for more information.


In [39]:
# !ls --help

In [45]:
import os
for file in os.listdir("cats_bigger_than_64x64"):
    if file.endswith(".jpg"):
        continue
    else:
        print(file)

In [46]:
# !rm -r cats_bigger_than_64x64/.ipynb_checkpoints